In [ ]:
%reload_ext autoreload
%load_ext blackcellmagic
%autoreload 2

import os

from lib.plot_utils import *
from lib.cdf_utils import *
from lib.scatter_utils import *

In [ ]:
csv_dir = os.path.join(CSV_ROOT_DIR, "total-flow")
query_str = 'problem != "one-wan-bidir.json" and problem != "msft-8075.json"'

In [ ]:
path_df = (
    pd.read_csv(os.path.join(csv_dir, "path-form-total_flow-slice_0_1_2_3_4.csv"))
    .drop(columns=["num_nodes", "num_edges", "num_commodities"])
    .query(PF_PARAMS)
)
path_df = sort_and_set_index(path_df)
if query_str is not None:
    path_df = path_df.query(query_str)

# POP DF
pop_df = pd.read_csv(os.path.join(csv_dir, "pop-total_flow-slice_0_1_2_3_4-k_16.csv"))
pop_df = sort_and_set_index(pop_df, drop=True)
if query_str is not None:
    pop_df = pop_df.query(query_str)
pop_df = pop_df.query(
    '(split_fraction == 0.0 and tm_model != "poisson-high-intra") or (split_fraction == 0.75 and tm_model == "poisson-high-intra")'
)

# NC Iterative DF
ncflow_df = per_iter_to_nc_df(
    os.path.join(csv_dir, "ncflow-total_flow-slices_0_1_2_3_4.csv")
)
if query_str is not None:
    ncflow_df = ncflow_df.query(query_str)


In [ ]:
ncflow_vs_pop_df = get_ratio_df(ncflow_df, pop_df, "obj_val", "_nc")

In [ ]:
ncflow_vs_pop_df.query('obj_val_ratio < 1.0 & speedup_ratio < 1.0')

In [ ]:
ncflow_vs_pop_df.query('obj_val_ratio > 1.0 & speedup_ratio > 1.0').sort_values('speedup_ratio')

In [ ]:
ncflow_vs_pop_df.query('obj_val_ratio > 1.0').sort_values('obj_val_ratio', ascending=False).head(20)

In [ ]:
ncflow_vs_pop_df.query('problem == "Kdl.graphml"').sort_values('speedup_ratio')

In [ ]:
ncflow_df.query('traffic_seed == 1857279712')[['obj_val', 'runtime']]

In [ ]:
pop_df.query('traffic_seed == 1857279712')[['obj_val', 'runtime']]

In [ ]:
ncflow_vs_pop_df.sort_values('obj_val_ratio')

In [ ]:
ncflow_vs_pop_df.sort_values('speedup_ratio')

## Scatter Plot of Total Flow

In [ ]:
def pop_random_scatter(path_form_csv, pop_csv, query_str):
    with_kdl = "problem == 'Kdl.graphml'"
    no_kdl = "problem != 'Kdl.graphml'"
    ratio_dfs = get_ratio_dataframes(
        path_form_csv,
        pop_csv,
        pop_parent_query_str=query_str,
        pop_query_strs=[with_kdl, no_kdl],
    )
    plot_dfs = [ratio_dfs["POP"][with_kdl], ratio_dfs["POP"][no_kdl]]
    scatter_plot(
        plot_dfs,
        ["KDL", "Non-KDL"],
        ["nc", "smore"],
        "speedup_ratio",
        "obj_val_ratio",
        xlim=None,
        ylim=[0.0, 1.1],
        figsize=(7.5, 4),
        xlabel="Speedup rel. to original problem",
        ylabel="Total flow rel. to\n original problem",
        bbta=(0, 0, 1, 2.35),
        ncol=2,
        xlog=True,
        save=True,
    )

In [ ]:
path_form_csv = 'max_total_flow_csvs/path-form-total-flow-slices_0_1_2_3_4.csv'
pop_csv = 'max_total_flow_csvs/pop-total_flow-slice_0-k_16.csv'

In [ ]:
pop_random_scatter(
    path_form_csv,
    pop_csv,
    "split_method == 'random' and num_subproblems == 16 and (tm_model == 'poisson-high-intra' and split_fraction == 0.75) or (tm_model != 'poisson-high-intra' and split_fraction == 0.0)",
)

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'poisson-high-inter'", "Poisson, High Inter")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'uniform'", "Uniform")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'gravity'", "Gravity")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   "tm_model == 'bimodal'", "Bimodal")

In [ ]:
pop_random_scatter(path_form_csv, pop_csv,
                   None, "all")